In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import rasterio as rio
import folium
import cv2 as cv
from datetime import datetime, timedelta

In [12]:
weather_dict = {}
for file in os.listdir('./eie_data/cleaned_gfs/'):
    weather_dict[file.split("_")[2].replace("XX.tif", "")[0:8]] = file

In [ ]:


def plot_scaled(file_name):
    vmin, vmax = np.nanpercentile(file_name, (5,95))  # 5-95% stretch
    img_plt = plt.imshow(file_name, cmap='gray', vmin=vmin, vmax=vmax)
    plt.show()

def increment_name(day, month, year):
    day += 1
    if day > 31:
        day = 1
        month += 1
        if month > 12:
            month = 1
            year += 1
    return day, month, year
    

patch_size = (3,3)
dir_list = os.listdir('./eie_data/cleaned_s5p_no2/')
dir_gfs_list = os.listdir('./eie_data/cleaned_gfs/')
dir_list = sorted(dir_list)
path = './eie_data/cleaned_s5p_no2/'
length = len(dir_list)
first_run = True
im_to_include = 10
for i, file in enumerate(dir_list):
    if i != length-1:
        image_feature = rio.open(path+file)
        shape_feature = image_feature.read(2)
        NO2_patch_feature = image.extract_patches_2d(shape_feature, patch_size)
        NO2_patch_feature = np.array([i.reshape(patch_size[0]*patch_size[1]) for i in NO2_patch_feature])
        week_features = NO2_patch_feature
        image_label = rio.open(path+dir_list[i+1])
        shape_label = image_label.read(2)
        NO2_patch_label = image.extract_patches_2d(shape_label, patch_size)
        NO2_patch_label = np.array([i.reshape(patch_size[0]*patch_size[1]) for i in NO2_patch_label])
        if first_run == True:
            labels = NO2_patch_label
        else:
            labels = np.append(labels, NO2_patch_label, axis=0)
        split_file_name = file.split("_")
        begin_period = split_file_name[3][0:8]
        end_period = split_file_name[4].replace(".tif", "")[0:8]
        day = int(begin_period[-2:])
        month = int(begin_period[-4:-2])
        year = int(begin_period[0:-4])
        date = datetime(year,month,day)
        for j in range(0,7):
            year_str = str(date.year)
            month_str = str(date.month)
            day_str = str(date.day)
            if len(month_str) < 2:
                month_str = "0" + month_str
            if len(day_str) < 2:
                day_str = "0" + day_str
            filename = year_str+month_str+day_str
            date += timedelta(days=1)
            path_gfs = './eie_data/cleaned_gfs/'+weather_dict[filename]
            image_feature = rio.open(path_gfs)
            for q in range (1, 7):
                weather_patch_feature = image.extract_patches_2d(image_feature.read(q), patch_size)
                weather_patch_feature = np.array([i.reshape(patch_size[0]*patch_size[1]) for i in weather_patch_feature])
                week_features = np.concatenate((week_features, weather_patch_feature), axis=1)
        if first_run == True:
            features = week_features
        else:
            features = np.concatenate((features, week_features), axis=0)
    if file == "cleaned_s5p_no2_20190623T161923_20190629T180700.tif":
        break
    first_run = False
    if i > im_to_include-2:
        break

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/image.py:287: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  indexing_strides = arr[slices].strides


In [12]:
print(labels.shape)
labels = np.array([label[int((patch_size[0]*patch_size[1]/2))] for label in labels])
print(labels.shape)
print(features.shape)

(138116, 9)
(138116,)
(138116, 387)


In [68]:
from sklearn.linear_model import Lasso
labels = pd.read_csv("labels.csv").drop(["Unnamed: 0"], axis=1)
features = pd.read_csv("features_images.csv").drop(["Unnamed: 0"], axis=1)
clf = Lasso(alpha=0.0000005)
clf.fit(features, labels)
print(clf.coef_)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000

In [48]:
#import pandas as pd 
#pd.DataFrame(features).to_csv("features_images.csv")
#pd.DataFrame(labels).to_csv("labels.csv")

In [69]:
from sklearn.feature_extraction import image
patch_size = (3,3)
dir_list = os.listdir('./eie_data/cleaned_s5p_no2/')
dir_gfs_list = os.listdir('./eie_data/cleaned_gfs/')
dir_list = sorted(dir_list)
path = './eie_data/cleaned_s5p_no2/'
length = len(dir_list)
first_run = True

for i, file in enumerate(dir_list):
    if i != length-1 and i >10:
        image_feature = rio.open(path+file)
        shape_feature = image_feature.read(2)
        NO2_patch_feature = image.extract_patches_2d(shape_feature, patch_size)
        NO2_patch_feature = np.array([i.reshape(patch_size[0]*patch_size[1]) for i in NO2_patch_feature])
        week_features = NO2_patch_feature
        image_label = rio.open(path+dir_list[i+1])
        shape_label = image_label.read(2)
        NO2_patch_label = image.extract_patches_2d(shape_label, patch_size)
        NO2_patch_label = np.array([i.reshape(patch_size[0]*patch_size[1]) for i in NO2_patch_label])
        if first_run == True:
            labels = NO2_patch_label
        else:
            labels = np.append(features, NO2_patch_label, axis=0)
        split_file_name = file.split("_")
        begin_period = split_file_name[3][0:8]
        end_period = split_file_name[4].replace(".tif", "")[0:8]
        day = int(begin_period[-2:])
        month = int(begin_period[-4:-2])
        year = int(begin_period[0:-4])
        date = datetime(year,month,day)
        for j in range(0,7):
            year_str = str(date.year)
            month_str = str(date.month)
            day_str = str(date.day)
            if len(month_str) < 2:
                month_str = "0" + month_str
            if len(day_str) < 2:
                day_str = "0" + day_str
            filename = year_str+month_str+day_str
            date += timedelta(days=1)
            path_gfs = './eie_data/cleaned_gfs/'+weather_dict[filename]
            image_feature = rio.open(path_gfs)
            for q in range (1, 7):
                weather_patch_feature = image.extract_patches_2d(image_feature.read(q), patch_size)
                weather_patch_feature = np.array([i.reshape(patch_size[0]*patch_size[1]) for i in weather_patch_feature])
                week_features = np.concatenate((week_features, weather_patch_feature), axis=1)
        if first_run == True:
            features = week_features
        else:
            features = np.concatenate(features, week_features, axis=0)
        first_run = False
    if file == "cleaned_s5p_no2_20190623T161923_20190629T180700.tif":
        break
   
    if i > 10:
        break

In [66]:
predictions = clf.predict(features)
#print(features[0].reshape(-1,1))
print(clf.predict(features[10000].reshape(1,-1)))
#labels = np.array([label[int((patch_size[0]*patch_size[1]/2))] for label in labels])
print(predictions)
#print(labels)
print(np.sum(predictions-labels))

[3.65502211e-06]
[2.10253351e-06 2.10253351e-06 2.10253351e-06 ... 3.06908358e-07
 3.06908358e-07 3.06908358e-07]
-0.5219173714461678


### 